<a href="https://colab.research.google.com/github/fjadidi2001/DecompDenoise/blob/main/DD_Feb11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from skimage.util import random_noise
from skimage.transform import rescale

# --------------------------
# 1. Synthetic Data Generation
# --------------------------

def add_gaussian_noise(image, mean=0, sigma=25):
    """Add Gaussian noise to an image."""
    noisy = image + np.random.normal(mean, sigma, image.shape).astype(np.uint8)
    return np.clip(noisy, 0, 255)

def add_salt_pepper_noise(image, prob=0.05):
    """Add salt-and-pepper noise to an image."""
    return (random_noise(image, mode='s&p', amount=prob) * 255).astype(np.uint8)

def add_poisson_noise(image):
    """Add Poisson noise to an image."""
    return (random_noise(image, mode='poisson') * 255).astype(np.uint8)

# --------------------------
# 2. Anscombe Transform (Variance Stabilization)
# --------------------------

def anscombe_transform(image):
    """Apply Anscombe transform to stabilize noise variance."""
    return 2 * np.sqrt(image + 3/8).astype(np.float32)

def inverse_anscombe_transform(transformed_image):
    """Approximate inverse Anscombe transform."""
    return (transformed_image / 2)**2 - 3/8

# --------------------------
# 3. Gabor Filter (Multi-scale Feature Extraction)
# --------------------------

def apply_gabor_filter(image, frequency=0.6, theta=0, sigma=1.5):
    """Apply Gabor filter to extract multi-scale features."""
    kernel = cv2.getGaborKernel(
        ksize=(21, 21),
        sigma=sigma,
        theta=theta,
        lambd=1/frequency,
        gamma=0.5,
        psi=0,
        ktype=cv2.CV_32F
    )
    filtered = cv2.filter2D(image, cv2.CV_8UC3, kernel)
    return filtered

# --------------------------
# 4. Preprocessing Pipeline
# --------------------------

def preprocess_image(image_path, target_size=(256, 256)):
    """Full preprocessing pipeline: load, resize, denoise, and filter."""
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")

    # Resize to target size
    image = cv2.resize(image, target_size)

    # Convert to grayscale (optional)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply synthetic noise (example: Gaussian)
    noisy_image = add_gaussian_noise(image)

    # Anscombe transform
    anscombe_img = anscombe_transform(noisy_image)

    # Gabor filtering (example: horizontal orientation)
    gabor_filtered = apply_gabor_filter(noisy_image, theta=np.pi/4)

    return {
        "original": image,
        "noisy": noisy_image,
        "anscombe": anscombe_img,
        "gabor_filtered": gabor_filtered
    }

# --------------------------
# Example Usage
# --------------------------

if __name__ == "__main__":
    # Example: Process a sample image
    image_path = "sample_image.jpg"  # Replace with your image path

    # Run preprocessing
    results = preprocess_image(image_path)

    # Save results (optional)
    output_dir = "preprocessed_results"
    os.makedirs(output_dir, exist_ok=True)

    for key, img in results.items():
        cv2.imwrite(f"{output_dir}/{key}.jpg", img)

    print("Preprocessing completed. Results saved to:", output_dir)

ValueError: Failed to load image: sample_image.jpg